In [1]:
# ======================================================================
#  Δ_i from Redfin (weighted WLS, county×quarter FE, covariates, control=1–2 stories)
#  → Pre-period capitalization tests in MLS (unweighted OLS, county×quarter FE, cluster by assoc)
#  Exports ONLY LaTeX bodies for three outcomes; prints Δ distribution to console
# ======================================================================

import os, re
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ----------------------------- PATHS -----------------------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin_sirs.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_mls.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ----------------------------- CORE COLS -----------------------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'   # used to define control = 1–2 stories

# ----------------------------- WINDOWS -----------------------------
# Δ uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START_Q   = '2019Q1'
PRE_END_Q     = '2022Q1'
POST_START_Q  = '2025Q1'
POST_END_Q    = '2025Q4'

# MLS capitalization regressions use the **pre** window only
MLS_PRE_START_Q = PRE_START_Q
MLS_PRE_END_Q   = '2021Q4'

# ----------------------------- RISK FAMILY -----------------------------
# Choose exactly one flood-risk family (or None)
RISK_SOURCE = 'fema'          # options: 'fema', 'firststreet', None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

# ----------------------------- COVARIATES (Stage 1) -----------------------------
# Full amenities/ownership/management/age set + optional coast distance if present
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc',
    'three_miles_coast_assoc'   # included only if present
]

# ----------------------------- MLS OUTCOMES -----------------------------
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ----------------------------- EXCLUSIONS -----------------------------
EXCLUDE_MIAMI_BROWARD = True  # exclude Miami-Dade and Broward everywhere

# ----------------------------- HELPERS -----------------------------
def _to_qstr(x):
    """Return 'YYYYQx' string for quarter-like inputs (handles Stata %tq ints/floats)."""
    if pd.api.types.is_period_dtype(x):
        return x.astype(str).str.upper()
    try:
        return pd.PeriodIndex(x, freq='Q').astype(str).str.upper()
    except Exception:
        pass
    def q_to_str(v):
        if pd.isna(v): return np.nan
        v = int(v)
        base = pd.Period('1960Q1', freq='Q')
        p = base + v
        return str(p).upper()
    return pd.Series(x).map(q_to_str)

def _in_range(qs, start_q, end_q):
    """Boolean mask: qs within [start_q, end_q], comparing as quarterly Periods."""
    qp = pd.PeriodIndex(qs, freq='Q')
    return (qp >= pd.Period(start_q, freq='Q')) & (qp <= pd.Period(end_q, freq='Q'))

def _county_q_fe(df, county_col=COUNTY_COL, q_col=QUARTER_COL):
    combo = df[county_col].astype(str).str.upper().str.strip() + ' | ' + df[q_col].astype(str)
    fe = pd.get_dummies(combo, prefix='cq', drop_first=False)
    return fe

def _weighted_mean(x, w):
    x = np.asarray(x, dtype='float64')
    w = np.asarray(w, dtype='float64')
    m = np.isfinite(x) & np.isfinite(w) & (w > 0)
    if not m.any(): return np.nan
    return (x[m]*w[m]).sum() / w[m].sum()

def _stars(p):
    if p is None or not np.isfinite(p): return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def _coef_se_line(res, name, scale=1.0, dec=3):
    # Return "b***" and "(se)" with scaling
    try:
        b = float(res.params[name])*scale
        se = float(res.bse[name])*scale
        p  = float(res.pvalues[name])
        return f"{b:.{dec}f}{_stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def _exclude_mia_broward(df, county_col=COUNTY_COL):
    cty = df[county_col].astype(str).str.upper().str.strip()
    cty = cty.str.replace(r'\s*COUNTY$', '', regex=True)
    return df.loc[~cty.isin(['MIAMI-DADE','MIAMI DADE','BROWARD'])].copy()

# ----------------------------- LOAD REDFIN + WEIGHTS -----------------------------
rf = pd.read_stata(PATH_DTA_REDFIN)
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()

# harmonize quarter keys
rf[QUARTER_COL] = _to_qstr(rf[QUARTER_COL])
wg[QUARTER_COL] = _to_qstr(wg[QUARTER_COL])

# exclude Miami-Dade & Broward early
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = _exclude_mia_broward(rf, COUNTY_COL)

# merge weights (many-to-one)
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').replace([np.inf, -np.inf], np.nan).fillna(1.0)

# usable HOA obs
rf = rf[pd.to_numeric(rf[HOA_PSF_COL], errors='coerce') > 0].copy()
rf['ln_hoa_psf'] = np.log(pd.to_numeric(rf[HOA_PSF_COL], errors='coerce').astype(float))

# ----------------------------- RISK DUMMIES -----------------------------
risk_df = pd.DataFrame(index=rf.index)
if RISK_SOURCE == 'fema' and FEMA_COL in rf.columns:
    risk_df = pd.get_dummies(rf[FEMA_COL], prefix='fema', drop_first=True)
elif RISK_SOURCE == 'firststreet' and FS_COL in rf.columns:
    risk_df = pd.get_dummies(rf[FS_COL], prefix='firststreet', drop_first=True)

# covariates actually present
avail_covars = [c for c in COVARS if c in rf.columns]
X_cov = rf[avail_covars] if avail_covars else pd.DataFrame(index=rf.index)

# ----------------------------- Stage 1 WLS: ln(HOA/ft²) ~ covars + risk + county×quarter FE -----------------------------
fe_cq = _county_q_fe(rf, COUNTY_COL, QUARTER_COL)
X1 = pd.concat([fe_cq, X_cov, risk_df], axis=1)
for c in X1.columns:
    if not np.issubdtype(X1[c].dtype, np.number):
        X1[c] = pd.to_numeric(X1[c], errors='coerce')
X1 = sm.add_constant(X1, has_constant='add')

y1 = rf['ln_hoa_psf'].astype(float)
w1 = rf[W_COL].astype(float)

mask = y1.notna() & X1.notna().all(axis=1) & w1.notna()
res1 = sm.WLS(y1[mask], X1[mask], weights=w1[mask]).fit()
rf.loc[mask, 'resid'] = res1.resid

# ----------------------------- Build Δ_i (percent DiD vs COUNTY control = 1–2 stories) -----------------------------
if STORIES_COL not in rf.columns:
    raise KeyError(f"Missing '{STORIES_COL}' to identify 1–2 story controls.")

# control rows = stories <= 2
ctrl = rf[(pd.to_numeric(rf[STORIES_COL], errors='coerce') <= 2) & rf['resid'].notna()].copy()
ctrl_cq = (ctrl
           .groupby([COUNTY_COL, QUARTER_COL], as_index=False)
           .apply(lambda d: pd.Series({'rbar_c': _weighted_mean(d['resid'], d[W_COL])}))
          )

# attach county×quarter control mean
rf = rf.merge(ctrl_cq, on=[COUNTY_COL, QUARTER_COL], how='left')

# pre/post tags
rf['is_pre']  = _in_range(rf[QUARTER_COL], PRE_START_Q,  PRE_END_Q)
rf['is_post'] = _in_range(rf[QUARTER_COL], POST_START_Q, POST_END_Q)
rf['period']  = np.where(rf['is_pre'], 'pre', np.where(rf['is_post'], 'post', np.nan))

# association-level weighted residual means (association & control) in pre/post
g = rf[rf['period'].isin(['pre','post'])].groupby([ASSOC_COL, 'period'], as_index=False)
a_means = g.apply(lambda d: pd.Series({
    'a_bar': _weighted_mean(d['resid'],  d[W_COL]),
    'c_bar': _weighted_mean(d['rbar_c'], d[W_COL])
}))
wide = (a_means.pivot(index=ASSOC_COL, columns='period', values=['a_bar','c_bar'])
                 .reset_index())
wide.columns = [ASSOC_COL] + ['_'.join([a for a in c if a]) for c in wide.columns.tolist()[1:]]

for c in ['a_bar_pre','a_bar_post','c_bar_pre','c_bar_post']:
    if c not in wide.columns: wide[c] = np.nan

# Δ in percent (linear log approximation): 100 * ( (a_post-a_pre) - (c_post-c_pre) )
wide['delta_pct'] = 100.0 * ((wide['a_bar_post'] - wide['a_bar_pre']) - (wide['c_bar_post'] - wide['c_bar_pre']))
delta = wide[[ASSOC_COL,'delta_pct']].dropna().copy()

# ----------------------------- Δ distribution (print only) -----------------------------
print("\n[Δ distribution] Δ_i = 100 × [(ā_post − ā_pre) − (ĉ_post − ĉ_pre)] in residual log points (%, approx)")
print(delta['delta_pct'].describe())

# ============================================================
# Stage 2: MLS pre-period capitalization
# y ∈ { ln(list/ft²), ln(sold/ft²), ln(1+DOM) } ~ Δ (pp) + county×quarter FE
# Unweighted OLS, SE clustered by county
# ============================================================
mls = pd.read_stata(PATH_DTA_MLS).copy()
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3]
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# mirror county exclusion
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = _exclude_mia_broward(mls, COUNTY_COL)

# keep only pre window
mls_pre = mls[_in_range(mls[QUARTER_COL], MLS_PRE_START_Q, MLS_PRE_END_Q)].copy()

# merge Δ onto MLS
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# outcomes: logs
def _ln_pos(df, col_in, col_out):
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = _ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = _ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')
pre_dom  = mls_pre.copy()
pre_dom  = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# design matrix: Δ + county×quarter FE
def _design(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')
    fe = _county_q_fe(df, COUNTY_COL, QUARTER_COL)
    X = pd.concat([df[['delta_pct']], fe], axis=1)
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')
    return y.loc[m], Xc, df.loc[m, COUNTY_COL]

def _run(y_name, df, ylab):
    y, X, g = _design(df, y_name)
    if y.empty:
        print(f"[MLS] {ylab}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {ylab} on Δ (pp) + county×quarter FE, unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = _run('ln_list_psf', pre_list, 'ln(list/ft²)')
res_sold = _run('ln_sold_psf', pre_sold, 'ln(sold/ft²)')
res_dom  = _run('ln_dom1p',   pre_dom,  'ln(1+DOM)')

# ============================================================
# LaTeX BODY exporter: 3 outcomes in one table body (no tabular env)
# ============================================================
def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=3):
    """
    Write a LaTeX body with 1 row per regressor in var_rows
    and 3 columns: Listed Price, Sold Price, Days on Market.
    
    var_rows: list of (param_name, row_label, scale)
    col_labels: list of 3 column labels (strings)
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write("% empty\n")
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def _coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        b, se = _coef_se_line(res, name, scale=scale, dec=dec)
        return b, se

    def _safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def _safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # Header row: blank stub + three outcome labels
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # Main coefficient rows
    for name, label, scale in var_rows:
        b_list, se_list, b_dom, se_dom = _coef_or_blank(res_list, name, scale)[0], _coef_or_blank(res_list, name, scale)[1], None, None

        b1, se1 = _coef_or_blank(res_list, name, scale)
        b2, se2 = _coef_or_blank(res_sold, name, scale)
        b3, se3 = _coef_or_blank(res_dom,  name, scale)

        # coefficient row
        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        # standard error row
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # Fit stats
    r2_1, r2_2, r2_3 = _safe_r2(res_list), _safe_r2(res_sold), _safe_r2(res_dom)
    n1, n2, n3 = _safe_nobs(res_list), _safe_nobs(res_sold), _safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"County $\times$ Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")
    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers (what shows at top of the table inside tabular)
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_pre.tex'),
    rows_common,
    col_labels,
    notes=r"\multicolumn{4}{l}{County-clustered SE in parentheses. \sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\"
)

print("\n[Done] LaTeX body saved in exports/delta_all_vars/: cap_mls_delta_3outcomes_body_pre.tex")



[Δ distribution] Δ_i = 100 × [(ā_post − ā_pre) − (ĉ_post − ĉ_pre)] in residual log points (%, approx)
count    654.000000
mean       5.320265
std       28.044955
min     -111.559912
25%      -11.197738
50%        4.428384
75%       19.488479
max      150.407579
Name: delta_pct, dtype: float64

[MLS PRE] ln(list/ft²) on Δ (pp) + county×quarter FE, unweighted | n=1731 R²=0.216
                            OLS Regression Results                            
Dep. Variable:            ln_list_psf   R-squared:                       0.216
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     1.958
Date:                Wed, 28 Jan 2026   Prob (F-statistic):              0.179
Time:                        13:43:28   Log-Likelihood:                -1285.4
No. Observations:                1731   AIC:                             2983.
Df Residuals:                    1525   BIC:                        

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 206, but rank is 1
  warnings.warn('covariance of constraints does not have full '



[MLS PRE] ln(sold/ft²) on Δ (pp) + county×quarter FE, unweighted | n=1766 R²=0.198
                            OLS Regression Results                            
Dep. Variable:            ln_sold_psf   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     2.317
Date:                Wed, 28 Jan 2026   Prob (F-statistic):              0.145
Time:                        13:43:28   Log-Likelihood:                -1406.8
No. Observations:                1766   AIC:                             3228.
Df Residuals:                    1559   BIC:                             4361.
Df Model:                         206                                         
Covariance Type:              cluster                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 207, but rank is 1
  warnings.warn('covariance of constraints does not have full '



[MLS PRE] ln(1+DOM) on Δ (pp) + county×quarter FE, unweighted | n=1772 R²=0.304
                            OLS Regression Results                            
Dep. Variable:               ln_dom1p   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     39.15
Date:                Wed, 28 Jan 2026   Prob (F-statistic):           6.68e-06
Time:                        13:43:28   Log-Likelihood:                -3031.3
No. Observations:                1772   AIC:                             6477.
Df Residuals:                    1565   BIC:                             7611.
Df Model:                         206                                         
Covariance Type:              cluster                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 207, but rank is 1
  warnings.warn('covariance of constraints does not have full '


In [2]:
# ======================================================================
#  Δ_i from Redfin (weighted WLS, county×quarter FE, covariates, control=1–2 stories)
#  → Post-period capitalization tests in Redfin (unweighted OLS, county×quarter FE, cluster by county)
#  Exports ONLY LaTeX bodies for three outcomes; prints Δ distribution to console
# ======================================================================

import os, re
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ----------------------------- PATHS -----------------------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin_sirs.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_mls.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ----------------------------- CORE COLS -----------------------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'   # used to define control = 1–2 stories

# ----------------------------- WINDOWS -----------------------------
# Δ uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START_Q   = '2019Q1'
PRE_END_Q     = '2022Q1'
POST_START_Q  = '2025Q1'
POST_END_Q    = '2025Q4'

# MLS capitalization regressions use the **post** window only
MLS_PRE_START_Q = '2022Q3'
MLS_PRE_END_Q   = '2023Q3'

# ----------------------------- RISK FAMILY -----------------------------
# Choose exactly one flood-risk family (or None)
RISK_SOURCE = 'fema'          # options: 'fema', 'firststreet', None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

# ----------------------------- COVARIATES (Stage 1) -----------------------------
# Full amenities/ownership/management/age set + optional coast distance if present
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc',
    'three_miles_coast_assoc'   # included only if present
]

# ----------------------------- MLS OUTCOMES -----------------------------
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ----------------------------- EXCLUSIONS -----------------------------
EXCLUDE_MIAMI_BROWARD = True  # exclude Miami-Dade and Broward everywhere

# ----------------------------- HELPERS -----------------------------
def _to_qstr(x):
    """Return 'YYYYQx' string for quarter-like inputs (handles Stata %tq ints/floats)."""
    if pd.api.types.is_period_dtype(x):
        return x.astype(str).str.upper()
    try:
        return pd.PeriodIndex(x, freq='Q').astype(str).str.upper()
    except Exception:
        pass
    def q_to_str(v):
        if pd.isna(v): return np.nan
        v = int(v)
        base = pd.Period('1960Q1', freq='Q')
        p = base + v
        return str(p).upper()
    return pd.Series(x).map(q_to_str)

def _in_range(qs, start_q, end_q):
    """Boolean mask: qs within [start_q, end_q], comparing as quarterly Periods."""
    qp = pd.PeriodIndex(qs, freq='Q')
    return (qp >= pd.Period(start_q, freq='Q')) & (qp <= pd.Period(end_q, freq='Q'))

def _county_q_fe(df, county_col=COUNTY_COL, q_col=QUARTER_COL):
    combo = df[county_col].astype(str).str.upper().str.strip() + ' | ' + df[q_col].astype(str)
    fe = pd.get_dummies(combo, prefix='cq', drop_first=False)
    return fe

def _weighted_mean(x, w):
    x = np.asarray(x, dtype='float64')
    w = np.asarray(w, dtype='float64')
    m = np.isfinite(x) & np.isfinite(w) & (w > 0)
    if not m.any(): return np.nan
    return (x[m]*w[m]).sum() / w[m].sum()

def _stars(p):
    if p is None or not np.isfinite(p): return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def _coef_se_line(res, name, scale=1.0, dec=3):
    # Return "b***" and "(se)" with scaling
    try:
        b = float(res.params[name])*scale
        se = float(res.bse[name])*scale
        p  = float(res.pvalues[name])
        return f"{b:.{dec}f}{_stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def _exclude_mia_broward(df, county_col=COUNTY_COL):
    cty = df[county_col].astype(str).str.upper().str.strip()
    cty = cty.str.replace(r'\s*COUNTY$', '', regex=True)
    return df.loc[~cty.isin(['MIAMI-DADE','MIAMI DADE','BROWARD'])].copy()

# ----------------------------- LOAD REDFIN + WEIGHTS -----------------------------
rf = pd.read_stata(PATH_DTA_REDFIN)
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()

# harmonize quarter keys
rf[QUARTER_COL] = _to_qstr(rf[QUARTER_COL])
wg[QUARTER_COL] = _to_qstr(wg[QUARTER_COL])

# exclude Miami-Dade & Broward early
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = _exclude_mia_broward(rf, COUNTY_COL)

# merge weights (many-to-one)
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').replace([np.inf, -np.inf], np.nan).fillna(1.0)

# usable HOA obs
rf = rf[pd.to_numeric(rf[HOA_PSF_COL], errors='coerce') > 0].copy()
rf['ln_hoa_psf'] = np.log(pd.to_numeric(rf[HOA_PSF_COL], errors='coerce').astype(float))

# ----------------------------- RISK DUMMIES -----------------------------
risk_df = pd.DataFrame(index=rf.index)
if RISK_SOURCE == 'fema' and FEMA_COL in rf.columns:
    risk_df = pd.get_dummies(rf[FEMA_COL], prefix='fema', drop_first=True)
elif RISK_SOURCE == 'firststreet' and FS_COL in rf.columns:
    risk_df = pd.get_dummies(rf[FS_COL], prefix='firststreet', drop_first=True)

# covariates actually present
avail_covars = [c for c in COVARS if c in rf.columns]
X_cov = rf[avail_covars] if avail_covars else pd.DataFrame(index=rf.index)

# ----------------------------- Stage 1 WLS: ln(HOA/ft²) ~ covars + risk + county×quarter FE -----------------------------
fe_cq = _county_q_fe(rf, COUNTY_COL, QUARTER_COL)
X1 = pd.concat([fe_cq, X_cov, risk_df], axis=1)
for c in X1.columns:
    if not np.issubdtype(X1[c].dtype, np.number):
        X1[c] = pd.to_numeric(X1[c], errors='coerce')
X1 = sm.add_constant(X1, has_constant='add')

y1 = rf['ln_hoa_psf'].astype(float)
w1 = rf[W_COL].astype(float)

mask = y1.notna() & X1.notna().all(axis=1) & w1.notna()
res1 = sm.WLS(y1[mask], X1[mask], weights=w1[mask]).fit()
rf.loc[mask, 'resid'] = res1.resid

# ----------------------------- Build Δ_i (percent DiD vs COUNTY control = 1–2 stories) -----------------------------
if STORIES_COL not in rf.columns:
    raise KeyError(f"Missing '{STORIES_COL}' to identify 1–2 story controls.")

# control rows = stories <= 2
ctrl = rf[(pd.to_numeric(rf[STORIES_COL], errors='coerce') <= 2) & rf['resid'].notna()].copy()
ctrl_cq = (ctrl
           .groupby([COUNTY_COL, QUARTER_COL], as_index=False)
           .apply(lambda d: pd.Series({'rbar_c': _weighted_mean(d['resid'], d[W_COL])}))
          )

# attach county×quarter control mean
rf = rf.merge(ctrl_cq, on=[COUNTY_COL, QUARTER_COL], how='left')

# pre/post tags
rf['is_pre']  = _in_range(rf[QUARTER_COL], PRE_START_Q,  PRE_END_Q)
rf['is_post'] = _in_range(rf[QUARTER_COL], POST_START_Q, POST_END_Q)
rf['period']  = np.where(rf['is_pre'], 'pre', np.where(rf['is_post'], 'post', np.nan))

# association-level weighted residual means (association & control) in pre/post
g = rf[rf['period'].isin(['pre','post'])].groupby([ASSOC_COL, 'period'], as_index=False)
a_means = g.apply(lambda d: pd.Series({
    'a_bar': _weighted_mean(d['resid'],  d[W_COL]),
    'c_bar': _weighted_mean(d['rbar_c'], d[W_COL])
}))
wide = (a_means.pivot(index=ASSOC_COL, columns='period', values=['a_bar','c_bar'])
                 .reset_index())
wide.columns = [ASSOC_COL] + ['_'.join([a for a in c if a]) for c in wide.columns.tolist()[1:]]

for c in ['a_bar_pre','a_bar_post','c_bar_pre','c_bar_post']:
    if c not in wide.columns: wide[c] = np.nan

# Δ in percent (linear log approximation): 100 * ( (a_post-a_pre) - (c_post-c_pre) )
wide['delta_pct'] = 100.0 * ((wide['a_bar_post'] - wide['a_bar_pre']) - (wide['c_bar_post'] - wide['c_bar_pre']))
delta = wide[[ASSOC_COL,'delta_pct']].dropna().copy()

# ----------------------------- Δ distribution (print only) -----------------------------
print("\n[Δ distribution] Δ_i = 100 × [(ā_post − ā_pre) − (ĉ_post − ĉ_pre)] in residual log points (%, approx)")
print(delta['delta_pct'].describe())

# ============================================================
# Stage 2: Redfin post-period capitalization (2025)
# y ∈ { ln(list/ft²), ln(sold/ft²), ln(1+DOM) } ~ Δ (pp) + county×quarter FE
# Unweighted OLS, SE clustered by county
# ============================================================
mls = pd.read_stata(PATH_DTA_MLS).copy()
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3]
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# mirror county exclusion
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = _exclude_mia_broward(mls, COUNTY_COL)

# keep only pre window
mls_pre = mls[_in_range(mls[QUARTER_COL], MLS_PRE_START_Q, MLS_PRE_END_Q)].copy()

# merge Δ onto MLS
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# outcomes: logs
def _ln_pos(df, col_in, col_out):
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = _ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = _ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')
pre_dom  = mls_pre.copy()
pre_dom  = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# design matrix: Δ + county×quarter FE
def _design(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')
    fe = _county_q_fe(df, COUNTY_COL, QUARTER_COL)
    X = pd.concat([df[['delta_pct']], fe], axis=1)
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')
    return y.loc[m], Xc, df.loc[m, COUNTY_COL]

def _run(y_name, df, ylab):
    y, X, g = _design(df, y_name)
    if y.empty:
        print(f"[MLS] {ylab}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {ylab} on Δ (pp) + county×quarter FE, unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = _run('ln_list_psf', pre_list, 'ln(list/ft²)')
res_sold = _run('ln_sold_psf', pre_sold, 'ln(sold/ft²)')
res_dom  = _run('ln_dom1p',   pre_dom,  'ln(1+DOM)')

# ============================================================
# LaTeX BODY exporter: 3 outcomes in one table body (no tabular env)
# ============================================================
def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=3):
    """
    Write a LaTeX body with 1 row per regressor in var_rows
    and 3 columns: Listed Price, Sold Price, Days on Market.
    
    var_rows: list of (param_name, row_label, scale)
    col_labels: list of 3 column labels (strings)
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write("% empty\n")
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def _coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        b, se = _coef_se_line(res, name, scale=scale, dec=dec)
        return b, se

    def _safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def _safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # Header row: blank stub + three outcome labels
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # Main coefficient rows
    for name, label, scale in var_rows:
        b_list, se_list, b_dom, se_dom = _coef_or_blank(res_list, name, scale)[0], _coef_or_blank(res_list, name, scale)[1], None, None

        b1, se1 = _coef_or_blank(res_list, name, scale)
        b2, se2 = _coef_or_blank(res_sold, name, scale)
        b3, se3 = _coef_or_blank(res_dom,  name, scale)

        # coefficient row
        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        # standard error row
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # Fit stats
    r2_1, r2_2, r2_3 = _safe_r2(res_list), _safe_r2(res_sold), _safe_r2(res_dom)
    n1, n2, n3 = _safe_nobs(res_list), _safe_nobs(res_sold), _safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"County $\times$ Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")
    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers (what shows at top of the table inside tabular)
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_post.tex'),
    rows_common,
    col_labels,
    notes=r"\multicolumn{4}{l}{County-clustered SE in parentheses. \sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\"
)

print("\n[Done] LaTeX body saved in exports/delta_all_vars/: cap_mls_delta_3outcomes_body_post.tex")



[Δ distribution] Δ_i = 100 × [(ā_post − ā_pre) − (ĉ_post − ĉ_pre)] in residual log points (%, approx)
count    654.000000
mean       5.320265
std       28.044955
min     -111.559912
25%      -11.197738
50%        4.428384
75%       19.488479
max      150.407579
Name: delta_pct, dtype: float64

[MLS PRE] ln(list/ft²) on Δ (pp) + county×quarter FE, unweighted | n=732 R²=0.219
                            OLS Regression Results                            
Dep. Variable:            ln_list_psf   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.119
Method:                 Least Squares   F-statistic:                     3.658
Date:                Wed, 28 Jan 2026   Prob (F-statistic):             0.0719
Time:                        13:43:35   Log-Likelihood:                -486.83
No. Observations:                 732   AIC:                             1142.
Df Residuals:                     648   BIC:                         

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 84, but rank is 1
  warnings.warn('covariance of constraints does not have full '
C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 80, but rank is 1
  warnings.warn('covariance of constraints does not have full '
C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 85, but rank is 1
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:               ln_dom1p   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     6.442
Date:                Wed, 28 Jan 2026   Prob (F-statistic):             0.0206
Time:                        13:43:36   Log-Likelihood:                -1225.6
No. Observations:                 742   AIC:                             2621.
Df Residuals:                     657   BIC:                             3013.
Df Model:                          84                                         
Covariance Type:              cluster                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [3]:
# ======================================================================
#  Δ_i from Redfin (weighted WLS, zip + quarter FE, covariates, control=1–2 stories)
#  → Pre-period capitalization tests in MLS (unweighted OLS, zip + quarter FE, cluster by assoc)
#  Exports ONLY LaTeX bodies for three outcomes; prints Δ distribution to console
# ======================================================================

import os, re
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ----------------------------- PATHS -----------------------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin_sirs.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_mls.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ----------------------------- CORE COLS -----------------------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
ZIP_COL     = 'zip5_attom'          # <--- NEW: zip-level identifier
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'   # used to define control = 1–2 stories

# ----------------------------- WINDOWS -----------------------------
# Δ uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START_Q   = '2019Q1'
PRE_END_Q     = '2022Q1'
POST_START_Q  = '2025Q1'
POST_END_Q    = '2025Q4'

# MLS capitalization regressions use the **pre** window only
MLS_PRE_START_Q = PRE_START_Q
MLS_PRE_END_Q   = '2021Q4'

# ----------------------------- RISK FAMILY -----------------------------
# Choose exactly one flood-risk family (or None)
RISK_SOURCE = 'fema'          # options: 'fema', 'firststreet', None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

# ----------------------------- COVARIATES (Stage 1) -----------------------------
# Full amenities/ownership/management/age set
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc'
]

# ----------------------------- MLS OUTCOMES -----------------------------
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ----------------------------- EXCLUSIONS -----------------------------
EXCLUDE_MIAMI_BROWARD = True  # exclude Miami-Dade and Broward everywhere

# ----------------------------- HELPERS -----------------------------
def _to_qstr(x):
    """Return 'YYYYQx' string for quarter-like inputs (handles Stata %tq ints/floats)."""
    if pd.api.types.is_period_dtype(x):
        return x.astype(str).str.upper()
    try:
        return pd.PeriodIndex(x, freq='Q').astype(str).str.upper()
    except Exception:
        pass
    def q_to_str(v):
        if pd.isna(v): return np.nan
        v = int(v)
        base = pd.Period('1960Q1', freq='Q')
        p = base + v
        return str(p).upper()
    return pd.Series(x).map(q_to_str)

def _in_range(qs, start_q, end_q):
    """Boolean mask: qs within [start_q, end_q], comparing as quarterly Periods."""
    qp = pd.PeriodIndex(qs, freq='Q')
    return (qp >= pd.Period(start_q, freq='Q')) & (qp <= pd.Period(end_q, freq='Q'))

def _county_q_fe(df, county_col=COUNTY_COL, q_col=QUARTER_COL):
    """(Old) County × quarter FE, kept here in case you still need it elsewhere."""
    combo = df[county_col].astype(str).str.upper().str.strip() + ' | ' + df[q_col].astype(str)
    fe = pd.get_dummies(combo, prefix='cq', drop_first=False)
    return fe

def _zip_quarter_fe(df, zip_col=ZIP_COL, q_col=QUARTER_COL):
    """
    Two-way fixed effects: zip and quarter separately (additive).
    Returns concatenated dummies for zip and quarter.
    """
    zip_series = df[zip_col].astype(str).str.upper().str.strip()
    q_series   = df[q_col].astype(str)
    fe_zip = pd.get_dummies(zip_series, prefix='zip', drop_first=False)
    fe_q   = pd.get_dummies(q_series,   prefix='q',   drop_first=False)
    return pd.concat([fe_zip, fe_q], axis=1)

def _weighted_mean(x, w):
    x = np.asarray(x, dtype='float64')
    w = np.asarray(w, dtype='float64')
    m = np.isfinite(x) & np.isfinite(w) & (w > 0)
    if not m.any(): return np.nan
    return (x[m]*w[m]).sum() / w[m].sum()

def _stars(p):
    if p is None or not np.isfinite(p): return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def _coef_se_line(res, name, scale=1.0, dec=3):
    # Return "b***" and "(se)" with scaling
    try:
        b = float(res.params[name])*scale
        se = float(res.bse[name])*scale
        p  = float(res.pvalues[name])
        return f"{b:.{dec}f}{_stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def _exclude_mia_broward(df, county_col=COUNTY_COL):
    cty = df[county_col].astype(str).str.upper().str.strip()
    cty = cty.str.replace(r'\s*COUNTY$', '', regex=True)
    return df.loc[~cty.isin(['MIAMI-DADE','MIAMI DADE','BROWARD'])].copy()

# ----------------------------- LOAD REDFIN + WEIGHTS -----------------------------
rf = pd.read_stata(PATH_DTA_REDFIN)
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()

# harmonize quarter keys
rf[QUARTER_COL] = _to_qstr(rf[QUARTER_COL])
wg[QUARTER_COL] = _to_qstr(wg[QUARTER_COL])

# exclude Miami-Dade & Broward early
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = _exclude_mia_broward(rf, COUNTY_COL)

# merge weights (many-to-one)
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').replace([np.inf, -np.inf], np.nan).fillna(1.0)

# usable HOA obs
rf = rf[pd.to_numeric(rf[HOA_PSF_COL], errors='coerce') > 0].copy()
rf['ln_hoa_psf'] = np.log(pd.to_numeric(rf[HOA_PSF_COL], errors='coerce').astype(float))

# ----------------------------- RISK DUMMIES -----------------------------
risk_df = pd.DataFrame(index=rf.index)
if RISK_SOURCE == 'fema' and FEMA_COL in rf.columns:
    risk_df = pd.get_dummies(rf[FEMA_COL], prefix='fema', drop_first=True)
elif RISK_SOURCE == 'firststreet' and FS_COL in rf.columns:
    risk_df = pd.get_dummies(rf[FS_COL], prefix='firststreet', drop_first=True)

# covariates actually present
avail_covars = [c for c in COVARS if c in rf.columns]
X_cov = rf[avail_covars] if avail_covars else pd.DataFrame(index=rf.index)

# ----------------------------- Stage 1 WLS: ln(HOA/ft²) ~ covars + risk + zip FE + quarter FE -----------------------------
fe_zq = _zip_quarter_fe(rf, ZIP_COL, QUARTER_COL)
X1 = pd.concat([fe_zq, X_cov, risk_df], axis=1)
for c in X1.columns:
    if not np.issubdtype(X1[c].dtype, np.number):
        X1[c] = pd.to_numeric(X1[c], errors='coerce')
X1 = sm.add_constant(X1, has_constant='add')

y1 = rf['ln_hoa_psf'].astype(float)
w1 = rf[W_COL].astype(float)

mask = y1.notna() & X1.notna().all(axis=1) & w1.notna()
res1 = sm.WLS(y1[mask], X1[mask], weights=w1[mask]).fit()
rf.loc[mask, 'resid'] = res1.resid

# ----------------------------- Build Δ_i (percent DiD vs COUNTY control = 1–2 stories) -----------------------------
if STORIES_COL not in rf.columns:
    raise KeyError(f"Missing '{STORIES_COL}' to identify 1–2 story controls.")

# control rows = stories <= 2
ctrl = rf[(pd.to_numeric(rf[STORIES_COL], errors='coerce') <= 2) & rf['resid'].notna()].copy()
ctrl_cq = (ctrl
           .groupby([COUNTY_COL, QUARTER_COL], as_index=False)
           .apply(lambda d: pd.Series({'rbar_c': _weighted_mean(d['resid'], d[W_COL])}))
          )

# attach county×quarter control mean
rf = rf.merge(ctrl_cq, on=[COUNTY_COL, QUARTER_COL], how='left')

# pre/post tags
rf['is_pre']  = _in_range(rf[QUARTER_COL], PRE_START_Q,  PRE_END_Q)
rf['is_post'] = _in_range(rf[QUARTER_COL], POST_START_Q, POST_END_Q)
rf['period']  = np.where(rf['is_pre'], 'pre', np.where(rf['is_post'], 'post', np.nan))

# association-level weighted residual means (association & control) in pre/post
g = rf[rf['period'].isin(['pre','post'])].groupby([ASSOC_COL, 'period'], as_index=False)
a_means = g.apply(lambda d: pd.Series({
    'a_bar': _weighted_mean(d['resid'],  d[W_COL]),
    'c_bar': _weighted_mean(d['rbar_c'], d[W_COL])
}))
wide = (a_means.pivot(index=ASSOC_COL, columns='period', values=['a_bar','c_bar'])
                 .reset_index())
wide.columns = [ASSOC_COL] + ['_'.join([a for a in c if a]) for c in wide.columns.tolist()[1:]]

for c in ['a_bar_pre','a_bar_post','c_bar_pre','c_bar_post']:
    if c not in wide.columns: wide[c] = np.nan

# Δ in percent (linear log approximation): 100 * ( (a_post-a_pre) - (c_post-c_pre) )
wide['delta_pct'] = 100.0 * ((wide['a_bar_post'] - wide['a_bar_pre']) - (wide['c_bar_post'] - wide['c_bar_pre']))
delta = wide[[ASSOC_COL,'delta_pct']].dropna().copy()

# ----------------------------- Δ distribution (print only) -----------------------------
print("\n[Δ distribution] Δ_i = 100 × [(ā_post − ā_pre) − (ĉ_post − ĉ_pre)] in residual log points (%, approx)")
print(delta['delta_pct'].describe())

# ============================================================
# Stage 2: MLS pre-period capitalization
# y ∈ { ln(list/ft²), ln(sold/ft²), ln(1+DOM) } ~ Δ (pp) + zip FE + quarter FE
# Unweighted OLS, SE clustered by county
# ============================================================
mls = pd.read_stata(PATH_DTA_MLS).copy()
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3]
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# mirror county exclusion
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = _exclude_mia_broward(mls, COUNTY_COL)

# keep only pre window
mls_pre = mls[_in_range(mls[QUARTER_COL], MLS_PRE_START_Q, MLS_PRE_END_Q)].copy()

# merge Δ onto MLS
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# outcomes: logs
def _ln_pos(df, col_in, col_out):
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = _ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = _ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')
pre_dom  = mls_pre.copy()
pre_dom  = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# design matrix: Δ + zip FE + quarter FE
def _design(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')
    fe = _zip_quarter_fe(df, ZIP_COL, QUARTER_COL)
    X = pd.concat([df[['delta_pct']], fe], axis=1)
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')
    return y.loc[m], Xc, df.loc[m, ZIP_COL]

def _run(y_name, df, ylab):
    y, X, g = _design(df, y_name)
    if y.empty:
        print(f"[MLS] {ylab}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {ylab} on Δ (pp) + zip FE + quarter FE, unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = _run('ln_list_psf', pre_list, 'ln(list/ft²)')
res_sold = _run('ln_sold_psf', pre_sold, 'ln(sold/ft²)')
res_dom  = _run('ln_dom1p',   pre_dom,  'ln(1+DOM)')

# ============================================================
# LaTeX BODY exporter: 3 outcomes in one table body (no tabular env)
# ============================================================
def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=3):
    """
    Write a LaTeX body with 1 row per regressor in var_rows
    and 3 columns: Listed Price, Sold Price, Days on Market.
    
    var_rows: list of (param_name, row_label, scale)
    col_labels: list of 3 column labels (strings)
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write("% empty\n")
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def _coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        b, se = _coef_se_line(res, name, scale=scale, dec=dec)
        return b, se

    def _safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def _safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # Header row: blank stub + three outcome labels
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # Main coefficient rows
    for name, label, scale in var_rows:
        b1, se1 = _coef_or_blank(res_list, name, scale)
        b2, se2 = _coef_or_blank(res_sold, name, scale)
        b3, se3 = _coef_or_blank(res_dom,  name, scale)

        # coefficient row
        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        # standard error row
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # Fit stats
    r2_1, r2_2, r2_3 = _safe_r2(res_list), _safe_r2(res_sold), _safe_r2(res_dom)
    n1, n2, n3 = _safe_nobs(res_list), _safe_nobs(res_sold), _safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"Zip and Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")
    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers (what shows at top of the table inside tabular)
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_pre.tex'),
    rows_common,
    col_labels,
    notes=r"\multicolumn{4}{l}{County-clustered SE in parentheses. \sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\"
)

print("\n[Done] LaTeX body saved in exports/delta_all_vars/: cap_mls_delta_3outcomes_body_pre.tex")



[Δ distribution] Δ_i = 100 × [(ā_post − ā_pre) − (ĉ_post − ĉ_pre)] in residual log points (%, approx)
count    654.000000
mean       5.869248
std       27.130803
min     -113.168349
25%       -9.478781
50%        5.208709
75%       20.182687
max      155.933305
Name: delta_pct, dtype: float64

[MLS PRE] ln(list/ft²) on Δ (pp) + zip FE + quarter FE, unweighted | n=1731 R²=0.714
                            OLS Regression Results                            
Dep. Variable:            ln_list_psf   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     119.1
Date:                Wed, 28 Jan 2026   Prob (F-statistic):           1.56e-57
Time:                        13:43:42   Log-Likelihood:                -411.64
No. Observations:                1731   AIC:                             1071.
Df Residuals:                    1607   BIC:                      

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 125, but rank is 12
  warnings.warn('covariance of constraints does not have full '
C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 125, but rank is 12
  warnings.warn('covariance of constraints does not have full '



[MLS PRE] ln(1+DOM) on Δ (pp) + zip FE + quarter FE, unweighted | n=1772 R²=0.305
                            OLS Regression Results                            
Dep. Variable:               ln_dom1p   R-squared:                       0.305
Model:                            OLS   Adj. R-squared:                  0.253
Method:                 Least Squares   F-statistic:                     43.91
Date:                Wed, 28 Jan 2026   Prob (F-statistic):           1.58e-36
Time:                        13:43:42   Log-Likelihood:                -3029.4
No. Observations:                1772   AIC:                             6307.
Df Residuals:                    1648   BIC:                             6986.
Df Model:                         123                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 125, but rank is 12
  warnings.warn('covariance of constraints does not have full '


In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# -------------------------------------------------
# Helper functions
# -------------------------------------------------

def q_to_start(q):
    """
    Convert quarter-like data to quarter-start timestamps.
    Accepts Stata tq, strings like '2019Q1', or dates.
    """
    # If already Period with Q freq
    if pd.api.types.is_period_dtype(q):
        return q.to_timestamp(how='start')
    dt = pd.to_datetime(q, errors='coerce')
    return pd.PeriodIndex(dt, freq='Q').to_timestamp(how='start')


def wmean(x, w):
    """Weighted mean with NaN / nonpositive weight handling."""
    x = np.asarray(x, dtype="float64")
    w = np.asarray(w, dtype="float64")
    m = np.isfinite(x) & np.isfinite(w) & (w > 0)
    if not m.any():
        return np.nan
    return (x[m] * w[m]).sum() / w[m].sum()


def exclude_mia_broward(df, county_col):
    """Drop Miami-Dade and Broward (county name column)."""
    cty = (df[county_col].astype(str).str.upper().str.strip()
           .str.replace(r'\s*COUNTY$', '', regex=True))
    return df.loc[~cty.isin(['MIAMI-DADE', 'MIAMI DADE', 'BROWARD'])].copy()


# -------------------------------------------------
# SHARED Δ FUNCTION
# -------------------------------------------------

def compute_delta(
    rf,
    *,
    assoc_col,
    county_col,
    zip_col,
    q_col,
    stories_col,
    hoa_psf_col,
    w_col,
    covars,
    pre_start,
    pre_end,
    post_start,
    post_end,
    ctrl_story_max=2,
    risk_source=None,      # 'fema', 'firststreet', or None
    fema_col=None,
    fs_col=None,
    verbose=True
):
    """
    Compute association-level Δ_i in percentage points (pp).

    Exact definition:
        1. Run a weighted regression in the Redfin panel:

              ln(HOA/ft²)_a,q  ~  covariates_a,q
                                      + risk dummies
                                      + ZIP fixed effects
                                      + quarter fixed effects

           using WLS with weights = final_weight.

        2. Take the **residuals** from this regression.

        3. For each county c and quarter q, compute the weighted mean residual
           among the **control buildings** (1–2 stories):

              r̄_c,q = weighted mean of residuals among controls in county c, q.

        4. For each association i, compute *association* and *control* means
           in the pre and post windows:

              ā_i,pre  = w-avg residual for assoc i in pre window
              ā_i,post = w-avg residual for assoc i in post window
              c̄_i,pre  = w-avg r̄_c,q (county-control mean) for quarters
                           of assoc i in pre window
              c̄_i,post = same in post window

        5. Define

              Δ_i = 100 * [ (ā_i,post − ā_i,pre)
                           −(c̄_i,post − c̄_i,pre) ]

           i.e., percentage-point change in residual log HOA for i
           relative to the change for 1–2 story controls in the same counties.

    Returns
    -------
    delta : DataFrame with columns [assoc_col, 'delta_pct']
    res1  : statsmodels WLS results object from step (1).
    """
    rf = rf.copy()

    # --- quarters as quarter-start timestamps
    rf[q_col] = q_to_start(rf[q_col])

    # --- log HOA per sq ft
    rf['ln_hoa_psf'] = np.log(pd.to_numeric(rf[hoa_psf_col], errors='coerce'))

    # --- pre / post indicators
    rf['period'] = np.select(
        [
            (rf[q_col] >= pre_start) & (rf[q_col] <= pre_end),
            (rf[q_col] >= post_start) & (rf[q_col] <= post_end),
        ],
        ['pre', 'post'],
        default=np.nan
    )

    # --- which covariates / risk columns exist
    covars_present = [c for c in covars if c in rf.columns]

    risk_cols = []
    if risk_source == 'fema' and fema_col is not None and fema_col in rf.columns:
        risk_cols.append(fema_col)
    elif risk_source == 'firststreet' and fs_col is not None and fs_col in rf.columns:
        risk_cols.append(fs_col)

    base_cols = [assoc_col, county_col, zip_col, q_col,
                 'period', 'ln_hoa_psf', w_col, stories_col]
    needed = base_cols + covars_present + risk_cols

    # --- subset to pre/post rows with valid ln(HOA)
    r0 = rf.loc[
        rf['period'].isin(['pre', 'post']) & rf['ln_hoa_psf'].notna(),
        needed
    ].copy()

    if verbose:
        print(f"[Δ] Raw rows in pre/post windows: {len(r0):,}")

    # --- build X matrix for Stage 1: covariates + risk + ZIP FE + Q FE
    parts = []

    if covars_present:
        Xcov = r0[covars_present].copy()
        for c in Xcov.columns:
            Xcov[c] = pd.to_numeric(Xcov[c], errors='coerce')
        parts.append(Xcov)

    if risk_source == 'fema' and fema_col is not None and fema_col in r0.columns:
        parts.append(pd.get_dummies(r0[fema_col], prefix='fema', drop_first=True))
    elif risk_source == 'firststreet' and fs_col is not None and fs_col in r0.columns:
        parts.append(pd.get_dummies(r0[fs_col], prefix='fs', drop_first=True))

    # ZIP FE
    parts.append(
        pd.get_dummies(
            r0[zip_col].astype(str).str.upper().str.strip(),
            prefix='zip', drop_first=True
        )
    )
    # Quarter FE
    parts.append(pd.get_dummies(r0[q_col], prefix='q', drop_first=True))

    X = pd.concat(parts, axis=1)
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    X = sm.add_constant(X, has_constant='add')

    y = r0['ln_hoa_psf'].astype(float)
    w = pd.to_numeric(r0[w_col], errors='coerce').replace([np.inf, -np.inf], np.nan).fillna(1.0)

    ok = X.notna().all(axis=1) & y.notna() & w.notna()
    r1 = r0.loc[ok].copy()
    X1, y1, w1 = X.loc[ok], y.loc[ok], w.loc[ok]

    if verbose:
        print(f"[Δ] Rows used in WLS: {len(r1):,} (dropped {len(r0)-len(r1):,})")

    res1 = sm.WLS(y1, X1, weights=w1).fit()
    r1['resid'] = res1.resid

    # --- 1–2 story controls: county×quarter mean residuals
    is_ctrl = pd.to_numeric(r1[stories_col], errors='coerce') <= ctrl_story_max
    ctrl = r1.loc[is_ctrl].copy()
    if ctrl.empty:
        print("[Δ] WARNING: no controls; Δ will be empty.")
        return pd.DataFrame({assoc_col: [], 'delta_pct': []}), res1

    c_means = (ctrl
               .groupby([county_col, q_col], as_index=False)
               .apply(lambda d: pd.Series({'rbar_c': wmean(d['resid'], d[w_col])})))
    r1 = r1.merge(c_means, on=[county_col, q_col], how='left')

    # --- association-level pre/post means of own residual and control mean
    ac_means = (r1
        .groupby([assoc_col, 'period'])
        .apply(lambda d: pd.Series({
            'a_bar': wmean(d['resid'],   d[w_col]),
            'c_bar': wmean(d['rbar_c'], d[w_col])
        }))
        .reset_index()
    )

    wide = (ac_means
            .pivot(index=assoc_col, columns='period', values=['a_bar', 'c_bar'])
            .reset_index())

    # flatten column names
    wide.columns = [assoc_col] + [
        '_'.join([a for a in c if a]) for c in wide.columns.tolist()[1:]
    ]

    for c in ['a_bar_pre', 'a_bar_post', 'c_bar_pre', 'c_bar_post']:
        if c not in wide.columns:
            wide[c] = np.nan

    # require all four means to compute Δ
    wide = wide.loc[
        wide[['a_bar_pre', 'a_bar_post', 'c_bar_pre', 'c_bar_post']].notna().all(1)
    ].copy()

    if verbose:
        print(f"[Δ] Associations with both pre & post residual means + county controls: {len(wide):,}")

    wide['delta_pct'] = 100.0 * (
        (wide['a_bar_post'] - wide['a_bar_pre']) -
        (wide['c_bar_post'] - wide['c_bar_pre'])
    )

    delta = wide[[assoc_col, 'delta_pct']].copy()

    if verbose and not delta.empty:
        print("\n[Δ] Distribution (pp):")
        print(delta['delta_pct'].describe())

    return delta, res1


In [8]:
# ============================================================
# CAPITALIZATION: MLS prices/DOM on Δ_i  (shared Δ method) - post period
# ============================================================
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ---------------- PATHS ----------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_price_dom_assoc_quarter_mls_redfin.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------- CORE COLS ----------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
ZIP_COL     = 'zip5_attom'
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'

# MLS outcomes
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ---------------- WINDOWS ----------------
# Δ_i uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START   = pd.Timestamp('2019-01-01')
PRE_END     = pd.Timestamp('2022-03-31')   # through 2022Q1
POST_START  = pd.Timestamp('2025-01-01')
POST_END    = pd.Timestamp('2025-12-31')

# MLS capitalization regressions use this **post** window (your existing choice)
MLS_PRE_START_Q = '2022Q3'
MLS_PRE_END_Q   = '2024Q4'

# ---------------- COVARIATES & RISK ----------------
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc'
]

RISK_SOURCE = 'fema'          # 'fema', 'firststreet', or None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

EXCLUDE_MIAMI_BROWARD = True

# ---------------- Stage 1: load Redfin + weights, compute Δ_i ----------------
# Load Redfin HOA panel
rf = pd.read_stata(PATH_DTA_REDFIN)

# Load weights and harmonize quarter keys
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()
wg[QUARTER_COL] = q_to_start(wg[QUARTER_COL])
wg[W_COL] = pd.to_numeric(wg[W_COL], errors='coerce').fillna(0)

# Aggregate any duplicate assoc×quarter weights
if wg.duplicated([ASSOC_COL, QUARTER_COL]).any():
    wg = wg.groupby([ASSOC_COL, QUARTER_COL], as_index=False)[W_COL].sum()

# Harmonize quarters in rf and merge weights
rf[QUARTER_COL] = q_to_start(rf[QUARTER_COL])
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').fillna(1.0)

# Exclude Miami-Dade & Broward
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = exclude_mia_broward(rf, COUNTY_COL)

# Compute Δ_i with the shared method
delta, res_stage1 = compute_delta(
    rf,
    assoc_col=ASSOC_COL,
    county_col=COUNTY_COL,
    zip_col=ZIP_COL,
    q_col=QUARTER_COL,
    stories_col=STORIES_COL,
    hoa_psf_col=HOA_PSF_COL,
    w_col=W_COL,
    covars=COVARS,
    pre_start=PRE_START,
    pre_end=PRE_END,
    post_start=POST_START,
    post_end=POST_END,
    ctrl_story_max=2,
    risk_source=RISK_SOURCE,
    fema_col=FEMA_COL,
    fs_col=FS_COL,
    verbose=True
)
# delta has columns: [ASSOC_COL, 'delta_pct']
delta_out_path = os.path.join(OUTPUT_DIR, "delta_assoc.csv")

# Ensure clean ordering and no index
delta[[ASSOC_COL, 'delta_pct']].to_csv(
    delta_out_path,
    index=False
)

print(f"Exported delta_assoc to {delta_out_path}")

# ---------------- Stage 2: MLS capitalization regressions ----------------
def ensure_numeric_df(X):
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    return X

def _to_qstr(q):
    """Convert MLS quarter variable to 'YYYYQx' string."""
    return pd.PeriodIndex(q, freq='Q').astype(str).str.upper()

# Load MLS dataset
mls = pd.read_stata(PATH_DTA_MLS).copy()

# Restrict to treated group: 3+ story associations
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3].copy()
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# Exclude Miami-Dade & Broward to match Stage 1
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = exclude_mia_broward(mls, COUNTY_COL)

# Keep only the MLS pre window (your chosen capitalization window)
mls_pre = mls[
    (mls[QUARTER_COL] >= MLS_PRE_START_Q) &
    (mls[QUARTER_COL] <= MLS_PRE_END_Q)
].copy()

# Merge Δ_i onto MLS panel (inner join → only associations with Δ)
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# --- build log outcomes ---

def ln_pos(df, col_in, col_out):
    """Keep rows with col_in>0 and add log(col_in) column."""
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')

pre_dom = mls_pre.copy()
pre_dom = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# --- design matrix: Δ_i + ZIP FE + quarter FE ---

def design_cap(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')

    # ZIP FE
    fe_zip = pd.get_dummies(
        df[ZIP_COL].astype(str).str.upper().str.strip(),
        prefix='zip', drop_first=True
    )
    # Quarter FE (using the 'YYYYQx' strings)
    fe_q = pd.get_dummies(df[QUARTER_COL], prefix='q', drop_first=True)

    X = pd.concat([df[['delta_pct']], fe_zip, fe_q], axis=1)
    X = ensure_numeric_df(X)

    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')

    groups = df.loc[m, ZIP_COL].astype(str)  # cluster by ZIP
    return y.loc[m], Xc, groups

def run_cap(df, y_name, y_label):
    y, X, g = design_cap(df, y_name)
    if y.empty:
        print(f"[MLS] {y_label}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {y_label} on Δ_i (pp) + ZIP FE + quarter FE, "
          f"unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = run_cap(pre_list, 'ln_list_psf', 'ln(list/ft²)')
res_sold = run_cap(pre_sold, 'ln_sold_psf', 'ln(sold/ft²)')
res_dom  = run_cap(pre_dom,  'ln_dom1p',   'ln(1+DOM)')


# ---------------- LaTeX body exporter (3 outcomes, Δ_i row only) ----------------
def stars(p):
    if p is None or not np.isfinite(p):
        return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def coef_se_line(res, name, scale=1.0, dec=4):
    """Return ('b***', '(se)') for regressor `name`."""
    try:
        b = float(res.params[name]) * scale
        se = float(res.bse[name]) * scale
        p = float(res.pvalues[name])
        return f"{b:.{dec}f}{stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=4):
    """
    Write only the LaTeX body (no \\begin{tabular}) for 3 outcome columns:
    Listed Price, Sold Price, Days on Market.
    `var_rows` is [(param_name, row_label, scale), ...].
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write('% empty\n')
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        return coef_se_line(res, name, scale=scale, dec=dec)

    def safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # header row
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # coefficient rows
    for name, label, scale in var_rows:
        b1, se1 = coef_or_blank(res_list, name, scale)
        b2, se2 = coef_or_blank(res_sold, name, scale)
        b3, se3 = coef_or_blank(res_dom,  name, scale)

        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # fit stats
    r2_1, r2_2, r2_3 = safe_r2(res_list), safe_r2(res_sold), safe_r2(res_dom)
    n1, n2, n3       = safe_nobs(res_list), safe_nobs(res_sold), safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"ZIP and Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")

    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers in the LaTeX table
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

latex_path = os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_post.tex')
export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    latex_path,
    rows_common,
    col_labels,
    notes=(r"\multicolumn{4}{l}{ZIP-clustered SE in parentheses. "
           r"\sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\")
)

print("\n[Done] LaTeX body saved in", latex_path)


[Δ] Raw rows in pre/post windows: 14,009
[Δ] Rows used in WLS: 11,382 (dropped 2,627)
[Δ] Associations with both pre & post residual means + county controls: 1,276

[Δ] Distribution (pp):
count    1276.000000
mean        5.774647
std        30.759039
min      -270.261056
25%        -8.442632
50%         5.200342
75%        20.393092
max       182.796828
Name: delta_pct, dtype: float64
Exported delta_assoc to exports/delta_all_vars\delta_assoc.csv

[MLS PRE] ln(list/ft²) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=3632 R²=0.658
                            OLS Regression Results                            
Dep. Variable:            ln_list_psf   R-squared:                       0.658
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     2472.
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          3.65e-196
Time:                        13:46:00   Log-Likelihood:           

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 201, but rank is 10
  warnings.warn('covariance of constraints does not have full '



[MLS PRE] ln(sold/ft²) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=3252 R²=0.655


C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 201, but rank is 10
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:            ln_sold_psf   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     1924.
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          7.25e-186
Time:                        13:46:01   Log-Likelihood:                -977.40
No. Observations:                3252   AIC:                             2359.
Df Residuals:                    3050   BIC:                             3588.
Df Model:                         201                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1679      0.011    586.114      0.0

                            OLS Regression Results                            
Dep. Variable:               ln_dom1p   R-squared:                       0.180
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     288.0
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          1.90e-109
Time:                        13:46:01   Log-Likelihood:                -5395.2
No. Observations:                3382   AIC:                         1.119e+04
Df Residuals:                    3180   BIC:                         1.243e+04
Df Model:                         201                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1992      0.064     65.931      0.0

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 201, but rank is 10
  warnings.warn('covariance of constraints does not have full '


In [14]:
# ============================================================
# CAPITALIZATION: MLS prices/DOM on Δ_i  (shared Δ method) - post period; effective age
# ============================================================
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ---------------- PATHS ----------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_price_dom_assoc_quarter_mls_redfin.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------- CORE COLS ----------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
ZIP_COL     = 'zip5_attom'
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'

# MLS outcomes
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ---------------- WINDOWS ----------------
# Δ_i uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START   = pd.Timestamp('2019-01-01')
PRE_END     = pd.Timestamp('2022-03-31')   # through 2022Q1
POST_START  = pd.Timestamp('2025-01-01')
POST_END    = pd.Timestamp('2025-12-31')

# MLS capitalization regressions use this **post** window (your existing choice)
MLS_PRE_START_Q = '2022Q3'
MLS_PRE_END_Q   = '2024Q4'

# ---------------- COVARIATES & RISK ----------------
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr','property_age_effect_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc'
]

RISK_SOURCE = 'fema'          # 'fema', 'firststreet', or None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

EXCLUDE_MIAMI_BROWARD = True

# ---------------- Stage 1: load Redfin + weights, compute Δ_i ----------------
# Load Redfin HOA panel
rf = pd.read_stata(PATH_DTA_REDFIN)

# Load weights and harmonize quarter keys
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()
wg[QUARTER_COL] = q_to_start(wg[QUARTER_COL])
wg[W_COL] = pd.to_numeric(wg[W_COL], errors='coerce').fillna(0)

# Aggregate any duplicate assoc×quarter weights
if wg.duplicated([ASSOC_COL, QUARTER_COL]).any():
    wg = wg.groupby([ASSOC_COL, QUARTER_COL], as_index=False)[W_COL].sum()

# Harmonize quarters in rf and merge weights
rf[QUARTER_COL] = q_to_start(rf[QUARTER_COL])
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').fillna(1.0)

# Exclude Miami-Dade & Broward
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = exclude_mia_broward(rf, COUNTY_COL)

# Compute Δ_i with the shared method
delta, res_stage1 = compute_delta(
    rf,
    assoc_col=ASSOC_COL,
    county_col=COUNTY_COL,
    zip_col=ZIP_COL,
    q_col=QUARTER_COL,
    stories_col=STORIES_COL,
    hoa_psf_col=HOA_PSF_COL,
    w_col=W_COL,
    covars=COVARS,
    pre_start=PRE_START,
    pre_end=PRE_END,
    post_start=POST_START,
    post_end=POST_END,
    ctrl_story_max=2,
    risk_source=RISK_SOURCE,
    fema_col=FEMA_COL,
    fs_col=FS_COL,
    verbose=True
)
# delta has columns: [ASSOC_COL, 'delta_pct']
delta_out_path = os.path.join(OUTPUT_DIR, "delta_assoc_effect_age.csv")

# Ensure clean ordering and no index
delta[[ASSOC_COL, 'delta_pct']].to_csv(
    delta_out_path,
    index=False
)

print(f"Exported delta_assoc to {delta_out_path}")

# ---------------- Stage 2: MLS capitalization regressions ----------------
def ensure_numeric_df(X):
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    return X

def _to_qstr(q):
    """Convert MLS quarter variable to 'YYYYQx' string."""
    return pd.PeriodIndex(q, freq='Q').astype(str).str.upper()

# Load MLS dataset
mls = pd.read_stata(PATH_DTA_MLS).copy()

# Restrict to treated group: 3+ story associations
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3].copy()
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# Exclude Miami-Dade & Broward to match Stage 1
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = exclude_mia_broward(mls, COUNTY_COL)

# Keep only the MLS pre window (your chosen capitalization window)
mls_pre = mls[
    (mls[QUARTER_COL] >= MLS_PRE_START_Q) &
    (mls[QUARTER_COL] <= MLS_PRE_END_Q)
].copy()

# Merge Δ_i onto MLS panel (inner join → only associations with Δ)
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# --- build log outcomes ---

def ln_pos(df, col_in, col_out):
    """Keep rows with col_in>0 and add log(col_in) column."""
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')

pre_dom = mls_pre.copy()
pre_dom = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# --- design matrix: Δ_i + ZIP FE + quarter FE ---

def design_cap(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')

    # ZIP FE
    fe_zip = pd.get_dummies(
        df[ZIP_COL].astype(str).str.upper().str.strip(),
        prefix='zip', drop_first=True
    )
    # Quarter FE (using the 'YYYYQx' strings)
    fe_q = pd.get_dummies(df[QUARTER_COL], prefix='q', drop_first=True)

    X = pd.concat([df[['delta_pct']], fe_zip, fe_q], axis=1)
    X = ensure_numeric_df(X)

    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')

    groups = df.loc[m, ZIP_COL].astype(str)  # cluster by ZIP
    return y.loc[m], Xc, groups

def run_cap(df, y_name, y_label):
    y, X, g = design_cap(df, y_name)
    if y.empty:
        print(f"[MLS] {y_label}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {y_label} on Δ_i (pp) + ZIP FE + quarter FE, "
          f"unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = run_cap(pre_list, 'ln_list_psf', 'ln(list/ft²)')
res_sold = run_cap(pre_sold, 'ln_sold_psf', 'ln(sold/ft²)')
res_dom  = run_cap(pre_dom,  'ln_dom1p',   'ln(1+DOM)')


# ---------------- LaTeX body exporter (3 outcomes, Δ_i row only) ----------------
def stars(p):
    if p is None or not np.isfinite(p):
        return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def coef_se_line(res, name, scale=1.0, dec=4):
    """Return ('b***', '(se)') for regressor `name`."""
    try:
        b = float(res.params[name]) * scale
        se = float(res.bse[name]) * scale
        p = float(res.pvalues[name])
        return f"{b:.{dec}f}{stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=4):
    """
    Write only the LaTeX body (no \\begin{tabular}) for 3 outcome columns:
    Listed Price, Sold Price, Days on Market.
    `var_rows` is [(param_name, row_label, scale), ...].
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write('% empty\n')
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        return coef_se_line(res, name, scale=scale, dec=dec)

    def safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # header row
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # coefficient rows
    for name, label, scale in var_rows:
        b1, se1 = coef_or_blank(res_list, name, scale)
        b2, se2 = coef_or_blank(res_sold, name, scale)
        b3, se3 = coef_or_blank(res_dom,  name, scale)

        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # fit stats
    r2_1, r2_2, r2_3 = safe_r2(res_list), safe_r2(res_sold), safe_r2(res_dom)
    n1, n2, n3       = safe_nobs(res_list), safe_nobs(res_sold), safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"ZIP and Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")

    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers in the LaTeX table
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

latex_path = os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_post_effect.tex')
export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    latex_path,
    rows_common,
    col_labels,
    notes=(r"\multicolumn{4}{l}{ZIP-clustered SE in parentheses. "
           r"\sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\")
)

print("\n[Done] LaTeX body saved in", latex_path)

[Δ] Raw rows in pre/post windows: 14,009
[Δ] Rows used in WLS: 8,293 (dropped 5,716)
[Δ] Associations with both pre & post residual means + county controls: 956

[Δ] Distribution (pp):
count    956.000000
mean       6.536295
std       31.654013
min     -243.647679
25%       -9.419236
50%        5.750373
75%       21.179534
max      181.402417
Name: delta_pct, dtype: float64
Exported delta_assoc to exports/delta_all_vars\delta_assoc_effect_age.csv

[MLS PRE] ln(list/ft²) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=2645 R²=0.699
                            OLS Regression Results                            
Dep. Variable:            ln_list_psf   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     1616.
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          1.21e-163
Time:                        13:50:47   Log-Likelihood:           

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 181, but rank is 10
  warnings.warn('covariance of constraints does not have full '
C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 181, but rank is 10
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:            ln_sold_psf   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     1726.
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          4.64e-166
Time:                        13:50:47   Log-Likelihood:                -395.28
No. Observations:                2378   AIC:                             1155.
Df Residuals:                    2196   BIC:                             2205.
Df Model:                         181                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1855      0.012    517.439      0.0

[LaTeX] wrote exports/delta_all_vars\cap_mls_delta_3outcomes_body_post_effect.tex

[Done] LaTeX body saved in exports/delta_all_vars\cap_mls_delta_3outcomes_body_post_effect.tex


C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 181, but rank is 10
  warnings.warn('covariance of constraints does not have full '


In [11]:
# ============================================================
# CAPITALIZATION: MLS prices/DOM on Δ_i  (shared Δ method) - pre period
# ============================================================
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ---------------- PATHS ----------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_price_dom_assoc_quarter_mls_redfin.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------- CORE COLS ----------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
ZIP_COL     = 'zip5_attom'
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'

# MLS outcomes
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ---------------- WINDOWS ----------------
# Δ_i uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START   = pd.Timestamp('2019-01-01')
PRE_END     = pd.Timestamp('2022-03-31')   # through 2022Q1
POST_START  = pd.Timestamp('2025-01-01')
POST_END    = pd.Timestamp('2025-12-31')

# MLS capitalization regressions use this **pre** window (your existing choice)
MLS_PRE_START_Q = '2019Q1'
MLS_PRE_END_Q   = '2022Q1'

# ---------------- COVARIATES & RISK ----------------
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc'
]

RISK_SOURCE = 'fema'          # 'fema', 'firststreet', or None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

EXCLUDE_MIAMI_BROWARD = True

# ---------------- Stage 1: load Redfin + weights, compute Δ_i ----------------
# Load Redfin HOA panel
rf = pd.read_stata(PATH_DTA_REDFIN)

# Load weights and harmonize quarter keys
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()
wg[QUARTER_COL] = q_to_start(wg[QUARTER_COL])
wg[W_COL] = pd.to_numeric(wg[W_COL], errors='coerce').fillna(0)

# Aggregate any duplicate assoc×quarter weights
if wg.duplicated([ASSOC_COL, QUARTER_COL]).any():
    wg = wg.groupby([ASSOC_COL, QUARTER_COL], as_index=False)[W_COL].sum()

# Harmonize quarters in rf and merge weights
rf[QUARTER_COL] = q_to_start(rf[QUARTER_COL])
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').fillna(1.0)

# Exclude Miami-Dade & Broward
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = exclude_mia_broward(rf, COUNTY_COL)

# Compute Δ_i with the shared method
delta, res_stage1 = compute_delta(
    rf,
    assoc_col=ASSOC_COL,
    county_col=COUNTY_COL,
    zip_col=ZIP_COL,
    q_col=QUARTER_COL,
    stories_col=STORIES_COL,
    hoa_psf_col=HOA_PSF_COL,
    w_col=W_COL,
    covars=COVARS,
    pre_start=PRE_START,
    pre_end=PRE_END,
    post_start=POST_START,
    post_end=POST_END,
    ctrl_story_max=2,
    risk_source=RISK_SOURCE,
    fema_col=FEMA_COL,
    fs_col=FS_COL,
    verbose=True
)
# delta has columns: [ASSOC_COL, 'delta_pct']

# ---------------- Stage 2: MLS capitalization regressions ----------------
def ensure_numeric_df(X):
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    return X

def _to_qstr(q):
    """Convert MLS quarter variable to 'YYYYQx' string."""
    return pd.PeriodIndex(q, freq='Q').astype(str).str.upper()

# Load MLS dataset
mls = pd.read_stata(PATH_DTA_MLS).copy()

# Restrict to treated group: 3+ story associations
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3].copy()
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# Exclude Miami-Dade & Broward to match Stage 1
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = exclude_mia_broward(mls, COUNTY_COL)

# Keep only the MLS pre window (your chosen capitalization window)
mls_pre = mls[
    (mls[QUARTER_COL] >= MLS_PRE_START_Q) &
    (mls[QUARTER_COL] <= MLS_PRE_END_Q)
].copy()

# Merge Δ_i onto MLS panel (inner join → only associations with Δ)
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# --- build log outcomes ---

def ln_pos(df, col_in, col_out):
    """Keep rows with col_in>0 and add log(col_in) column."""
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')

pre_dom = mls_pre.copy()
pre_dom = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# --- design matrix: Δ_i + ZIP FE + quarter FE ---

def design_cap(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')

    # ZIP FE
    fe_zip = pd.get_dummies(
        df[ZIP_COL].astype(str).str.upper().str.strip(),
        prefix='zip', drop_first=True
    )
    # Quarter FE (using the 'YYYYQx' strings)
    fe_q = pd.get_dummies(df[QUARTER_COL], prefix='q', drop_first=True)

    X = pd.concat([df[['delta_pct']], fe_zip, fe_q], axis=1)
    X = ensure_numeric_df(X)

    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')

    groups = df.loc[m, ZIP_COL].astype(str)  # cluster by ZIP
    return y.loc[m], Xc, groups

def run_cap(df, y_name, y_label):
    y, X, g = design_cap(df, y_name)
    if y.empty:
        print(f"[MLS] {y_label}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {y_label} on Δ_i (pp) + ZIP FE + quarter FE, "
          f"unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = run_cap(pre_list, 'ln_list_psf', 'ln(list/ft²)')
res_sold = run_cap(pre_sold, 'ln_sold_psf', 'ln(sold/ft²)')
res_dom  = run_cap(pre_dom,  'ln_dom1p',   'ln(1+DOM)')


# ---------------- LaTeX body exporter (3 outcomes, Δ_i row only) ----------------
def stars(p):
    if p is None or not np.isfinite(p):
        return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def coef_se_line(res, name, scale=1.0, dec=4):
    """Return ('b***', '(se)') for regressor `name`."""
    try:
        b = float(res.params[name]) * scale
        se = float(res.bse[name]) * scale
        p = float(res.pvalues[name])
        return f"{b:.{dec}f}{stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=4):
    """
    Write only the LaTeX body (no \\begin{tabular}) for 3 outcome columns:
    Listed Price, Sold Price, Days on Market.
    `var_rows` is [(param_name, row_label, scale), ...].
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write('% empty\n')
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        return coef_se_line(res, name, scale=scale, dec=dec)

    def safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # header row
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # coefficient rows
    for name, label, scale in var_rows:
        b1, se1 = coef_or_blank(res_list, name, scale)
        b2, se2 = coef_or_blank(res_sold, name, scale)
        b3, se3 = coef_or_blank(res_dom,  name, scale)

        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # fit stats
    r2_1, r2_2, r2_3 = safe_r2(res_list), safe_r2(res_sold), safe_r2(res_dom)
    n1, n2, n3       = safe_nobs(res_list), safe_nobs(res_sold), safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"ZIP and Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")

    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers in the LaTeX table
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

latex_path = os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_pre.tex')
export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    latex_path,
    rows_common,
    col_labels,
    notes=(r"\multicolumn{4}{l}{ZIP-clustered SE in parentheses. "
           r"\sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\")
)

print("\n[Done] LaTeX body saved in", latex_path)


[Δ] Raw rows in pre/post windows: 14,009
[Δ] Rows used in WLS: 11,382 (dropped 2,627)
[Δ] Associations with both pre & post residual means + county controls: 1,276

[Δ] Distribution (pp):
count    1276.000000
mean        5.774647
std        30.759039
min      -270.261056
25%        -8.442632
50%         5.200342
75%        20.393092
max       182.796828
Name: delta_pct, dtype: float64

[MLS PRE] ln(list/ft²) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=5349 R²=0.606
                            OLS Regression Results                            
Dep. Variable:            ln_list_psf   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.590
Method:                 Least Squares   F-statistic:                     3568.
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          2.91e-218
Time:                        13:46:47   Log-Likelihood:                -2586.9
No. Observations:                5349   AIC:      

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 202, but rank is 13
  warnings.warn('covariance of constraints does not have full '



[MLS PRE] ln(sold/ft²) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=5468 R²=0.560
                            OLS Regression Results                            
Dep. Variable:            ln_sold_psf   R-squared:                       0.560
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     4032.
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          2.94e-223
Time:                        13:46:47   Log-Likelihood:                -3089.5
No. Observations:                5468   AIC:                             6585.
Df Residuals:                    5265   BIC:                             7926.
Df Model:                         202                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 202, but rank is 13
  warnings.warn('covariance of constraints does not have full '



[MLS PRE] ln(1+DOM) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=5479 R²=0.303
                            OLS Regression Results                            
Dep. Variable:               ln_dom1p   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.276
Method:                 Least Squares   F-statistic:                     121.8
Date:                Wed, 28 Jan 2026   Prob (F-statistic):           2.31e-84
Time:                        13:46:47   Log-Likelihood:                -9655.8
No. Observations:                5479   AIC:                         1.972e+04
Df Residuals:                    5276   BIC:                         2.106e+04
Df Model:                         202                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 202, but rank is 13
  warnings.warn('covariance of constraints does not have full '


In [15]:
# ============================================================
# CAPITALIZATION: MLS prices/DOM on Δ_i  (shared Δ method) - pre period; effective age
# ============================================================
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ---------------- PATHS ----------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_price_dom_assoc_quarter_mls_redfin.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------- CORE COLS ----------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
ZIP_COL     = 'zip5_attom'
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'

# MLS outcomes
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ---------------- WINDOWS ----------------
# Δ_i uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START   = pd.Timestamp('2019-01-01')
PRE_END     = pd.Timestamp('2022-03-31')   # through 2022Q1
POST_START  = pd.Timestamp('2025-01-01')
POST_END    = pd.Timestamp('2025-12-31')

# MLS capitalization regressions use this **pre** window (your existing choice)
MLS_PRE_START_Q = '2019Q1'
MLS_PRE_END_Q   = '2022Q1'

# ---------------- COVARIATES & RISK ----------------
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr' ,'property_age_effect_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc'
]

RISK_SOURCE = 'fema'          # 'fema', 'firststreet', or None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

EXCLUDE_MIAMI_BROWARD = True

# ---------------- Stage 1: load Redfin + weights, compute Δ_i ----------------
# Load Redfin HOA panel
rf = pd.read_stata(PATH_DTA_REDFIN)

# Load weights and harmonize quarter keys
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()
wg[QUARTER_COL] = q_to_start(wg[QUARTER_COL])
wg[W_COL] = pd.to_numeric(wg[W_COL], errors='coerce').fillna(0)

# Aggregate any duplicate assoc×quarter weights
if wg.duplicated([ASSOC_COL, QUARTER_COL]).any():
    wg = wg.groupby([ASSOC_COL, QUARTER_COL], as_index=False)[W_COL].sum()

# Harmonize quarters in rf and merge weights
rf[QUARTER_COL] = q_to_start(rf[QUARTER_COL])
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').fillna(1.0)

# Exclude Miami-Dade & Broward
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = exclude_mia_broward(rf, COUNTY_COL)

# Compute Δ_i with the shared method
delta, res_stage1 = compute_delta(
    rf,
    assoc_col=ASSOC_COL,
    county_col=COUNTY_COL,
    zip_col=ZIP_COL,
    q_col=QUARTER_COL,
    stories_col=STORIES_COL,
    hoa_psf_col=HOA_PSF_COL,
    w_col=W_COL,
    covars=COVARS,
    pre_start=PRE_START,
    pre_end=PRE_END,
    post_start=POST_START,
    post_end=POST_END,
    ctrl_story_max=2,
    risk_source=RISK_SOURCE,
    fema_col=FEMA_COL,
    fs_col=FS_COL,
    verbose=True
)
# delta has columns: [ASSOC_COL, 'delta_pct']

# ---------------- Stage 2: MLS capitalization regressions ----------------
def ensure_numeric_df(X):
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    return X

def _to_qstr(q):
    """Convert MLS quarter variable to 'YYYYQx' string."""
    return pd.PeriodIndex(q, freq='Q').astype(str).str.upper()

# Load MLS dataset
mls = pd.read_stata(PATH_DTA_MLS).copy()

# Restrict to treated group: 3+ story associations
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3].copy()
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# Exclude Miami-Dade & Broward to match Stage 1
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = exclude_mia_broward(mls, COUNTY_COL)

# Keep only the MLS pre window (your chosen capitalization window)
mls_pre = mls[
    (mls[QUARTER_COL] >= MLS_PRE_START_Q) &
    (mls[QUARTER_COL] <= MLS_PRE_END_Q)
].copy()

# Merge Δ_i onto MLS panel (inner join → only associations with Δ)
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# --- build log outcomes ---

def ln_pos(df, col_in, col_out):
    """Keep rows with col_in>0 and add log(col_in) column."""
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')

pre_dom = mls_pre.copy()
pre_dom = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# --- design matrix: Δ_i + ZIP FE + quarter FE ---

def design_cap(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')

    # ZIP FE
    fe_zip = pd.get_dummies(
        df[ZIP_COL].astype(str).str.upper().str.strip(),
        prefix='zip', drop_first=True
    )
    # Quarter FE (using the 'YYYYQx' strings)
    fe_q = pd.get_dummies(df[QUARTER_COL], prefix='q', drop_first=True)

    X = pd.concat([df[['delta_pct']], fe_zip, fe_q], axis=1)
    X = ensure_numeric_df(X)

    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')

    groups = df.loc[m, ZIP_COL].astype(str)  # cluster by ZIP
    return y.loc[m], Xc, groups

def run_cap(df, y_name, y_label):
    y, X, g = design_cap(df, y_name)
    if y.empty:
        print(f"[MLS] {y_label}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {y_label} on Δ_i (pp) + ZIP FE + quarter FE, "
          f"unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = run_cap(pre_list, 'ln_list_psf', 'ln(list/ft²)')
res_sold = run_cap(pre_sold, 'ln_sold_psf', 'ln(sold/ft²)')
res_dom  = run_cap(pre_dom,  'ln_dom1p',   'ln(1+DOM)')


# ---------------- LaTeX body exporter (3 outcomes, Δ_i row only) ----------------
def stars(p):
    if p is None or not np.isfinite(p):
        return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def coef_se_line(res, name, scale=1.0, dec=4):
    """Return ('b***', '(se)') for regressor `name`."""
    try:
        b = float(res.params[name]) * scale
        se = float(res.bse[name]) * scale
        p = float(res.pvalues[name])
        return f"{b:.{dec}f}{stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=4):
    """
    Write only the LaTeX body (no \\begin{tabular}) for 3 outcome columns:
    Listed Price, Sold Price, Days on Market.
    `var_rows` is [(param_name, row_label, scale), ...].
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write('% empty\n')
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        return coef_se_line(res, name, scale=scale, dec=dec)

    def safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # header row
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # coefficient rows
    for name, label, scale in var_rows:
        b1, se1 = coef_or_blank(res_list, name, scale)
        b2, se2 = coef_or_blank(res_sold, name, scale)
        b3, se3 = coef_or_blank(res_dom,  name, scale)

        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # fit stats
    r2_1, r2_2, r2_3 = safe_r2(res_list), safe_r2(res_sold), safe_r2(res_dom)
    n1, n2, n3       = safe_nobs(res_list), safe_nobs(res_sold), safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"ZIP and Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")

    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers in the LaTeX table
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

latex_path = os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_pre_effect.tex')
export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    latex_path,
    rows_common,
    col_labels,
    notes=(r"\multicolumn{4}{l}{ZIP-clustered SE in parentheses. "
           r"\sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\")
)

print("\n[Done] LaTeX body saved in", latex_path)

[Δ] Raw rows in pre/post windows: 14,009
[Δ] Rows used in WLS: 8,293 (dropped 5,716)
[Δ] Associations with both pre & post residual means + county controls: 956

[Δ] Distribution (pp):
count    956.000000
mean       6.536295
std       31.654013
min     -243.647679
25%       -9.419236
50%        5.750373
75%       21.179534
max      181.402417
Name: delta_pct, dtype: float64

[MLS PRE] ln(list/ft²) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=3915 R²=0.669
                            OLS Regression Results                            
Dep. Variable:            ln_list_psf   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     2899.
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          4.24e-191
Time:                        13:51:38   Log-Likelihood:                -1357.7
No. Observations:                3915   AIC:                 

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 182, but rank is 13
  warnings.warn('covariance of constraints does not have full '



[MLS PRE] ln(sold/ft²) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=3997 R²=0.652
                            OLS Regression Results                            
Dep. Variable:            ln_sold_psf   R-squared:                       0.652
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     3540.
Date:                Wed, 28 Jan 2026   Prob (F-statistic):          2.12e-198
Time:                        13:51:39   Log-Likelihood:                -1513.4
No. Observations:                3997   AIC:                             3393.
Df Residuals:                    3814   BIC:                             4544.
Df Model:                         182                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 182, but rank is 13
  warnings.warn('covariance of constraints does not have full '



[MLS PRE] ln(1+DOM) on Δ_i (pp) + ZIP FE + quarter FE, unweighted | n=4001 R²=0.305
                            OLS Regression Results                            
Dep. Variable:               ln_dom1p   R-squared:                       0.305
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     76.18
Date:                Wed, 28 Jan 2026   Prob (F-statistic):           1.54e-63
Time:                        13:51:39   Log-Likelihood:                -7080.8
No. Observations:                4001   AIC:                         1.453e+04
Df Residuals:                    3818   BIC:                         1.568e+04
Df Model:                         182                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------

C:\Users\ngodin\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 182, but rank is 13
  warnings.warn('covariance of constraints does not have full '


In [ ]:
# ============================================================
# CAPITALIZATION: MLS prices/DOM on Δ_i  (shared Δ method) - post period SIRS
# ============================================================
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ---------------- PATHS ----------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin_sirs.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_price_dom_assoc_quarter_mls_redfin.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------- CORE COLS ----------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
ZIP_COL     = 'zip5_attom'
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'

# MLS outcomes
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ---------------- WINDOWS ----------------
# Δ_i uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START   = pd.Timestamp('2019-01-01')
PRE_END     = pd.Timestamp('2022-03-31')   # through 2022Q1
POST_START  = pd.Timestamp('2025-01-01')
POST_END    = pd.Timestamp('2025-12-31')

# MLS capitalization regressions use this **post** window (your existing choice)
MLS_PRE_START_Q = '2022Q3'
MLS_PRE_END_Q   = '2024Q4'

# ---------------- COVARIATES & RISK ----------------
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc'
]

RISK_SOURCE = 'fema'          # 'fema', 'firststreet', or None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

EXCLUDE_MIAMI_BROWARD = True

# ---------------- Stage 1: load Redfin + weights, compute Δ_i ----------------
# Load Redfin HOA panel
rf = pd.read_stata(PATH_DTA_REDFIN)

# Load weights and harmonize quarter keys
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()
wg[QUARTER_COL] = q_to_start(wg[QUARTER_COL])
wg[W_COL] = pd.to_numeric(wg[W_COL], errors='coerce').fillna(0)

# Aggregate any duplicate assoc×quarter weights
if wg.duplicated([ASSOC_COL, QUARTER_COL]).any():
    wg = wg.groupby([ASSOC_COL, QUARTER_COL], as_index=False)[W_COL].sum()

# Harmonize quarters in rf and merge weights
rf[QUARTER_COL] = q_to_start(rf[QUARTER_COL])
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').fillna(1.0)

# Exclude Miami-Dade & Broward
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = exclude_mia_broward(rf, COUNTY_COL)

# Compute Δ_i with the shared method
delta, res_stage1 = compute_delta(
    rf,
    assoc_col=ASSOC_COL,
    county_col=COUNTY_COL,
    zip_col=ZIP_COL,
    q_col=QUARTER_COL,
    stories_col=STORIES_COL,
    hoa_psf_col=HOA_PSF_COL,
    w_col=W_COL,
    covars=COVARS,
    pre_start=PRE_START,
    pre_end=PRE_END,
    post_start=POST_START,
    post_end=POST_END,
    ctrl_story_max=2,
    risk_source=RISK_SOURCE,
    fema_col=FEMA_COL,
    fs_col=FS_COL,
    verbose=True
)
# delta has columns: [ASSOC_COL, 'delta_pct']

# ---------------- Stage 2: MLS capitalization regressions ----------------
def ensure_numeric_df(X):
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    return X

def _to_qstr(q):
    """Convert MLS quarter variable to 'YYYYQx' string."""
    return pd.PeriodIndex(q, freq='Q').astype(str).str.upper()

# Load MLS dataset
mls = pd.read_stata(PATH_DTA_MLS).copy()

# Restrict to treated group: 3+ story associations
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3].copy()
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# Exclude Miami-Dade & Broward to match Stage 1
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = exclude_mia_broward(mls, COUNTY_COL)

# Keep only the MLS pre window (your chosen capitalization window)
mls_pre = mls[
    (mls[QUARTER_COL] >= MLS_PRE_START_Q) &
    (mls[QUARTER_COL] <= MLS_PRE_END_Q)
].copy()

# Merge Δ_i onto MLS panel (inner join → only associations with Δ)
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# --- build log outcomes ---

def ln_pos(df, col_in, col_out):
    """Keep rows with col_in>0 and add log(col_in) column."""
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')

pre_dom = mls_pre.copy()
pre_dom = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# --- design matrix: Δ_i + ZIP FE + quarter FE ---

def design_cap(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')

    # ZIP FE
    fe_zip = pd.get_dummies(
        df[ZIP_COL].astype(str).str.upper().str.strip(),
        prefix='zip', drop_first=True
    )
    # Quarter FE (using the 'YYYYQx' strings)
    fe_q = pd.get_dummies(df[QUARTER_COL], prefix='q', drop_first=True)

    X = pd.concat([df[['delta_pct']], fe_zip, fe_q], axis=1)
    X = ensure_numeric_df(X)

    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')

    groups = df.loc[m, ZIP_COL].astype(str)  # cluster by ZIP
    return y.loc[m], Xc, groups

def run_cap(df, y_name, y_label):
    y, X, g = design_cap(df, y_name)
    if y.empty:
        print(f"[MLS] {y_label}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {y_label} on Δ_i (pp) + ZIP FE + quarter FE, "
          f"unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = run_cap(pre_list, 'ln_list_psf', 'ln(list/ft²)')
res_sold = run_cap(pre_sold, 'ln_sold_psf', 'ln(sold/ft²)')
res_dom  = run_cap(pre_dom,  'ln_dom1p',   'ln(1+DOM)')


# ---------------- LaTeX body exporter (3 outcomes, Δ_i row only) ----------------
def stars(p):
    if p is None or not np.isfinite(p):
        return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def coef_se_line(res, name, scale=1.0, dec=4):
    """Return ('b***', '(se)') for regressor `name`."""
    try:
        b = float(res.params[name]) * scale
        se = float(res.bse[name]) * scale
        p = float(res.pvalues[name])
        return f"{b:.{dec}f}{stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=4):
    """
    Write only the LaTeX body (no \\begin{tabular}) for 3 outcome columns:
    Listed Price, Sold Price, Days on Market.
    `var_rows` is [(param_name, row_label, scale), ...].
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write('% empty\n')
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        return coef_se_line(res, name, scale=scale, dec=dec)

    def safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # header row
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # coefficient rows
    for name, label, scale in var_rows:
        b1, se1 = coef_or_blank(res_list, name, scale)
        b2, se2 = coef_or_blank(res_sold, name, scale)
        b3, se3 = coef_or_blank(res_dom,  name, scale)

        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # fit stats
    r2_1, r2_2, r2_3 = safe_r2(res_list), safe_r2(res_sold), safe_r2(res_dom)
    n1, n2, n3       = safe_nobs(res_list), safe_nobs(res_sold), safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"ZIP and Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")

    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers in the LaTeX table
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

latex_path = os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_post_sirs.tex')
export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    latex_path,
    rows_common,
    col_labels,
    notes=(r"\multicolumn{4}{l}{ZIP-clustered SE in parentheses. "
           r"\sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\")
)

print("\n[Done] LaTeX body saved in", latex_path)

In [ ]:
# ============================================================
# CAPITALIZATION: MLS prices/DOM on Δ_i  (shared Δ method) - pre period SIRS
# ============================================================
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ---------------- PATHS ----------------
PATH_DTA_REDFIN = '../../final_datasets/master_datasets/master_dataset_assoc_quarter_redfin_sirs.dta'
PATH_WGTS       = '../../final_datasets/master_datasets/hoa_redfin_weights.dta'
PATH_DTA_MLS    = '../../final_datasets/master_datasets/master_dataset_price_dom_assoc_quarter_mls_redfin.dta'
OUTPUT_DIR      = 'exports/delta_all_vars'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------- CORE COLS ----------------
ASSOC_COL   = 'assoc_name_final'
COUNTY_COL  = 'mm_fips_county_name_attom'
ZIP_COL     = 'zip5_attom'
QUARTER_COL = 'quarter'
HOA_PSF_COL = 'hoa_sq_ft_assoc_qtr'
W_COL       = 'final_weight'
STORIES_COL = 'num_stories_final_assoc'

# MLS outcomes
MLS_LIST_PSF = 'list_price_sq_ft_assoc_qtr'
MLS_SOLD_PSF = 'sold_price_sq_ft_assoc_qtr'
MLS_DOM      = 'dom_assoc_qtr'

# ---------------- WINDOWS ----------------
# Δ_i uses 2019–2022Q1 (pre) vs 2025 (post), consistent with lending spec
PRE_START   = pd.Timestamp('2019-01-01')
PRE_END     = pd.Timestamp('2022-03-31')   # through 2022Q1
POST_START  = pd.Timestamp('2025-01-01')
POST_END    = pd.Timestamp('2025-12-31')

# MLS capitalization regressions use this **pre** window (your existing choice)
MLS_PRE_START_Q = '2019Q1'
MLS_PRE_END_Q   = '2022Q1'

# ---------------- COVARIATES & RISK ----------------
COVARS = [
    'gym_redfin_assoc','pool_redfin_assoc','spa_broad_redfin_assoc',
    'tennis_redfin_assoc','golf_redfin_assoc','garage_redfin_assoc',
    'boat_redfin_assoc','elevator_redfin_assoc','view_redfin_assoc',
    'senior_community_redfin_assoc','property_age_assoc_qtr',
    'frac_npexcorp_state_attom_assoc','frac_corp_own_attom_assoc',
    'corp_mgmt_city_attom_assoc'
]

RISK_SOURCE = 'fema'          # 'fema', 'firststreet', or None
FEMA_COL    = 'fema_flood_risk_bucket_assoc'
FS_COL      = 'firststreet_risk_cat_assoc'

EXCLUDE_MIAMI_BROWARD = True

# ---------------- Stage 1: load Redfin + weights, compute Δ_i ----------------
# Load Redfin HOA panel
rf = pd.read_stata(PATH_DTA_REDFIN)

# Load weights and harmonize quarter keys
wg = pd.read_stata(PATH_WGTS)[[ASSOC_COL, QUARTER_COL, W_COL]].copy()
wg[QUARTER_COL] = q_to_start(wg[QUARTER_COL])
wg[W_COL] = pd.to_numeric(wg[W_COL], errors='coerce').fillna(0)

# Aggregate any duplicate assoc×quarter weights
if wg.duplicated([ASSOC_COL, QUARTER_COL]).any():
    wg = wg.groupby([ASSOC_COL, QUARTER_COL], as_index=False)[W_COL].sum()

# Harmonize quarters in rf and merge weights
rf[QUARTER_COL] = q_to_start(rf[QUARTER_COL])
rf = rf.merge(wg, on=[ASSOC_COL, QUARTER_COL], how='left', validate='m:1')
rf[W_COL] = pd.to_numeric(rf[W_COL], errors='coerce').fillna(1.0)

# Exclude Miami-Dade & Broward
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in rf.columns:
    rf = exclude_mia_broward(rf, COUNTY_COL)

# Compute Δ_i with the shared method
delta, res_stage1 = compute_delta(
    rf,
    assoc_col=ASSOC_COL,
    county_col=COUNTY_COL,
    zip_col=ZIP_COL,
    q_col=QUARTER_COL,
    stories_col=STORIES_COL,
    hoa_psf_col=HOA_PSF_COL,
    w_col=W_COL,
    covars=COVARS,
    pre_start=PRE_START,
    pre_end=PRE_END,
    post_start=POST_START,
    post_end=POST_END,
    ctrl_story_max=2,
    risk_source=RISK_SOURCE,
    fema_col=FEMA_COL,
    fs_col=FS_COL,
    verbose=True
)
# delta has columns: [ASSOC_COL, 'delta_pct']

# ---------------- Stage 2: MLS capitalization regressions ----------------
def ensure_numeric_df(X):
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors='coerce')
    return X

def _to_qstr(q):
    """Convert MLS quarter variable to 'YYYYQx' string."""
    return pd.PeriodIndex(q, freq='Q').astype(str).str.upper()

# Load MLS dataset
mls = pd.read_stata(PATH_DTA_MLS).copy()

# Restrict to treated group: 3+ story associations
mls = mls[pd.to_numeric(mls[STORIES_COL], errors='coerce') >= 3].copy()
mls[QUARTER_COL] = _to_qstr(mls[QUARTER_COL])

# Exclude Miami-Dade & Broward to match Stage 1
if EXCLUDE_MIAMI_BROWARD and COUNTY_COL in mls.columns:
    mls = exclude_mia_broward(mls, COUNTY_COL)

# Keep only the MLS pre window (your chosen capitalization window)
mls_pre = mls[
    (mls[QUARTER_COL] >= MLS_PRE_START_Q) &
    (mls[QUARTER_COL] <= MLS_PRE_END_Q)
].copy()

# Merge Δ_i onto MLS panel (inner join → only associations with Δ)
mls_pre = mls_pre.merge(delta, on=ASSOC_COL, how='inner')

# --- build log outcomes ---

def ln_pos(df, col_in, col_out):
    """Keep rows with col_in>0 and add log(col_in) column."""
    x = pd.to_numeric(df[col_in], errors='coerce')
    df = df[x > 0].copy()
    df[col_out] = np.log(x[x > 0].astype(float))
    return df

pre_list = ln_pos(mls_pre.copy(), MLS_LIST_PSF, 'ln_list_psf')
pre_sold = ln_pos(mls_pre.copy(), MLS_SOLD_PSF, 'ln_sold_psf')

pre_dom = mls_pre.copy()
pre_dom = pre_dom[pd.to_numeric(pre_dom[MLS_DOM], errors='coerce') >= 0].copy()
pre_dom['ln_dom1p'] = np.log1p(pd.to_numeric(pre_dom[MLS_DOM], errors='coerce').astype(float))

# --- design matrix: Δ_i + ZIP FE + quarter FE ---

def design_cap(df, ycol):
    y = pd.to_numeric(df[ycol], errors='coerce')

    # ZIP FE
    fe_zip = pd.get_dummies(
        df[ZIP_COL].astype(str).str.upper().str.strip(),
        prefix='zip', drop_first=True
    )
    # Quarter FE (using the 'YYYYQx' strings)
    fe_q = pd.get_dummies(df[QUARTER_COL], prefix='q', drop_first=True)

    X = pd.concat([df[['delta_pct']], fe_zip, fe_q], axis=1)
    X = ensure_numeric_df(X)

    m = y.notna() & X.notna().all(axis=1)
    Xc = sm.add_constant(X.loc[m], has_constant='add')

    groups = df.loc[m, ZIP_COL].astype(str)  # cluster by ZIP
    return y.loc[m], Xc, groups

def run_cap(df, y_name, y_label):
    y, X, g = design_cap(df, y_name)
    if y.empty:
        print(f"[MLS] {y_label}: no usable observations.")
        return None
    fit = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': g})
    print(f"\n[MLS PRE] {y_label} on Δ_i (pp) + ZIP FE + quarter FE, "
          f"unweighted | n={int(fit.nobs)} R²={fit.rsquared:.3f}")
    print(fit.summary())
    return fit

res_list = run_cap(pre_list, 'ln_list_psf', 'ln(list/ft²)')
res_sold = run_cap(pre_sold, 'ln_sold_psf', 'ln(sold/ft²)')
res_dom  = run_cap(pre_dom,  'ln_dom1p',   'ln(1+DOM)')


# ---------------- LaTeX body exporter (3 outcomes, Δ_i row only) ----------------
def stars(p):
    if p is None or not np.isfinite(p):
        return ''
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def coef_se_line(res, name, scale=1.0, dec=4):
    """Return ('b***', '(se)') for regressor `name`."""
    try:
        b = float(res.params[name]) * scale
        se = float(res.bse[name]) * scale
        p = float(res.pvalues[name])
        return f"{b:.{dec}f}{stars(p)}", f"({se:.{dec}f})"
    except Exception:
        return "", ""

def export_body_threecols(res_list, res_sold, res_dom,
                          out_path,
                          var_rows,
                          col_labels,
                          notes=None,
                          fe_markers=True,
                          dec=4):
    """
    Write only the LaTeX body (no \\begin{tabular}) for 3 outcome columns:
    Listed Price, Sold Price, Days on Market.
    `var_rows` is [(param_name, row_label, scale), ...].
    """
    # if all results are None, write an empty file
    if (res_list is None) and (res_sold is None) and (res_dom is None):
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write('% empty\n')
        print(f"[LaTeX] wrote empty {out_path}")
        return

    def coef_or_blank(res, name, scale):
        if res is None:
            return "", ""
        return coef_se_line(res, name, scale=scale, dec=dec)

    def safe_r2(res):
        try:
            return f"{res.rsquared:.3f}"
        except Exception:
            return ""

    def safe_nobs(res):
        try:
            return str(int(round(res.nobs)))
        except Exception:
            return ""

    lines = []

    # header row
    lines.append(" & " + " & ".join(col_labels) + r" \\")
    lines.append(r"\midrule")

    # coefficient rows
    for name, label, scale in var_rows:
        b1, se1 = coef_or_blank(res_list, name, scale)
        b2, se2 = coef_or_blank(res_sold, name, scale)
        b3, se3 = coef_or_blank(res_dom,  name, scale)

        lines.append(label + " & " + " & ".join([b1, b2, b3]) + r" \\")
        lines.append(" & " + " & ".join([se1, se2, se3]) + r" \\")
        lines.append(r"\addlinespace")

    # fit stats
    r2_1, r2_2, r2_3 = safe_r2(res_list), safe_r2(res_sold), safe_r2(res_dom)
    n1, n2, n3       = safe_nobs(res_list), safe_nobs(res_sold), safe_nobs(res_dom)

    lines.append(r"$R^2$ & " + " & ".join([r2_1, r2_2, r2_3]) + r" \\")
    lines.append("Observations & " + " & ".join([n1, n2, n3]) + r" \\")
    if fe_markers:
        fe_vals = ["Yes" if res is not None else "" for res in (res_list, res_sold, res_dom)]
        lines.append(r"ZIP and Quarter FE & " + " & ".join(fe_vals) + r" \\")

    if notes:
        lines.append(r"\midrule")
        lines.append(notes)

    with open(out_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines) + "\n")

    print(f"[LaTeX] wrote {out_path}")


# rows to report (just Δ_i)
rows_common = [('delta_pct', r'$\Delta_i$ (pp)', 1)]

# column headers in the LaTeX table
col_labels = [
    r'Listed Price',
    r'Sold Price',
    r'Days on Market'
]

latex_path = os.path.join(OUTPUT_DIR, 'cap_mls_delta_3outcomes_body_pre_sirs.tex')
export_body_threecols(
    res_list,
    res_sold,
    res_dom,
    latex_path,
    rows_common,
    col_labels,
    notes=(r"\multicolumn{4}{l}{ZIP-clustered SE in parentheses. "
           r"\sym{*} $p<0.10$, \sym{**} $p<0.05$, \sym{***} $p<0.01$} \\")
)

print("\n[Done] LaTeX body saved in", latex_path)